# ID3 Algorithm

## Pseudocode for ID3

    /* I is the set of input attributes
    * O is the output attribute
    * T is a set of training data
    *
    * function ID3 returns a decision tree
    */


    if (T is empty) {
    return a single node with the value "Failure";
    }

    if (all records in T have the same value for O) {
    return a single node with that value;
    }

    if (I is empty) {
    return a single node with the value of the most frequent value of
    O in T;

    /* Note: some elements in this node will be incorrectly classified */
    }
    /* now handle the case where we can’t return a single node */
    compute the information gain for each attribute in I relative to T;
    let X be the attribute with largest Gain(X, T) of the attributes in I;
    let {x_j| j=1,2, .., m} be the values of X;
    let {T_j| j=1,2, .., m} be the subsets of T when T is partitioned
    according the value of X;
    return a tree with the root node labelled X and
    arcs labelled x_1, x_2, .., x_m, where the arcs go to the
    trees ID3(I-{X}, O, T_1), ID3(I-{X}, O, T_2), .., ID3(I-{X}, O, T_m);
    }

### Code

In [0]:
import pandas as pd
import numpy as np
import math

In [0]:
df_w = pd.read_csv('weather.csv')
df_w

,Outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


Entropy Calculation: <br>
* grab the number of yes and now with a groupby
* calculate entropy of row
* sum over the rows

In [0]:
def H(data):
    H_T = 0 
    n = len(data) 
    # get the number of yes/no
    d = data.groupby(data.columns[-1],as_index=False).count().iloc[:,0:2] 
    # calc prob of yes/no
    prob = d[d.columns[-1]]/n
    # loop over each row and sum entropies
    for row in prob:
        H_T += row*np.log2(row)
    return -H_T
    

In [0]:
H(df_w)

0.9402859586706309

In [0]:
np.unique(df_w.values.T[1])

array(['cool', 'hot', 'mild'], dtype=object)

In [0]:
df_w.columns.values[:-1]

array(['Outlook', 'temperature', 'humidity', 'windy'], dtype=object)

Gain Calculation: <br>
* Subtracts the entropy of the dataset with the entropy of each attribute
* Append the calculation to a list and sort the list

In [0]:
def Gain(data):
    n = len(data)
    node = []
    gain = {}
    if H(data) == 0:
        return [data[data.columns[-1]].values[0],"leaf-node"]
    # loop over each column and grab counts of yes and no
    for column in data.columns.values[:-1]:
        values,count = np.unique(data[column].values,return_counts=True)
        H_values=[]
        # create a dataframe and apply H function
        for value in values:
            df_values = data[data[column] == value]
            np.array(H_values.append(H(df_values)))
        # sum the entropies and subtract from dependent entropy
        gainCalc = H(data) - np.sum((count/n)*H_values)
        # input entropies into a dictionary
        gain.update({column : gainCalc})
    # generate a list of attribute and values
    # originally also was a dictionary, but was becoming too difficult to work with   
    node.append(max(gain, key=gain.get))
    node.append(list(np.unique(data[max(gain, key=gain.get)])))
    return node

In [0]:
Gain(df_w)

['Outlook', ['overcast', 'rainy', 'sunny']]

In [0]:
Gain(df_w)[0]

'Outlook'

In [0]:
import random
vals = []
for i in range(20):
    test = []
    for col in df_w.columns[:-1]:
        test.append(random.choice(df_w[col].unique()))
    vals.append(test)
testdata = pd.DataFrame(vals,columns = df_w.columns[:-1])

In [0]:
testdata

,Outlook,temperature,humidity,windy
0,overcast,hot,high,True
1,sunny,cool,normal,True
2,overcast,mild,normal,True
3,overcast,hot,high,True
4,sunny,cool,high,False
5,rainy,hot,normal,False
6,rainy,cool,normal,True
7,rainy,mild,high,True
8,sunny,hot,normal,False
9,rainy,mild,normal,True


The classify function: <br>
* loop over the test dataframe row by row
* run the ID3 algorithm on each row
* append the leaf-node label onto the test dataframe
* repeat


In [0]:
def classify(train,test):
    # play attribute values to add
    play_label=[]
    # loop over each row
    for i in range(len(test)):
        traincopy=train
        output=Gain(traincopy)
        # leaf-node output is the label to be added
        # run ID3 until leaf-node is returned
        while output[1]!="leaf-node":
            print(output[1])
            # select entry to make new dataframe with
            dataentry=test.loc[i,output[0]]
            print(dataentry)
            # dataframe to run next iteration of ID3 over
            newtrain=traincopy.loc[train[output[0]]==dataentry]
            print(newtrain)
            output=Gain(newtrain)
            print(output)
            traincopy=newtrain
        # create a column of play values for test dataset
        play_label.append(output[0])
    # add the column
    test['label(play)']=np.array(play_label)
    return test

In [0]:
classify(df_w,testdata)

['overcast', 'rainy', 'sunny']
overcast
     Outlook temperature humidity  windy play
2   overcast         hot     high  False  yes
6   overcast        cool   normal   True  yes
11  overcast        mild     high   True  yes
12  overcast         hot   normal  False  yes
['yes', 'leaf-node']
['overcast', 'rainy', 'sunny']
sunny
   Outlook temperature humidity  windy play
0    sunny         hot     high  False   no
1    sunny         hot     high   True   no
7    sunny        mild     high  False   no
8    sunny        cool   normal  False  yes
10   sunny        mild   normal   True  yes
['humidity', ['high', 'normal']]
['high', 'normal']
normal
   Outlook temperature humidity  windy play
8    sunny        cool   normal  False  yes
10   sunny        mild   normal   True  yes
['yes', 'leaf-node']
['overcast', 'rainy', 'sunny']
overcast
     Outlook temperature humidity  windy play
2   overcast         hot     high  False  yes
6   overcast        cool   normal   True  yes
11  overcast      

,Outlook,temperature,humidity,windy,label(play)
0,overcast,hot,high,True,yes
1,sunny,cool,normal,True,yes
2,overcast,mild,normal,True,yes
3,overcast,hot,high,True,yes
4,sunny,cool,high,False,no
5,rainy,hot,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,high,True,no
8,sunny,hot,normal,False,yes
9,rainy,mild,normal,True,no
